In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [15]:
import sys
sys.path.append('./models') 
from gaussian import GP
from gaussian import RBF
from bandit import Bandit

sys.path.append('./train_data') 
from train_func import TrueFunc

In [17]:


sigma = 0.3
f_max = 0.5
is_func = True

In [18]:
expectation_func = TrueFunc(sigma, f_max, is_func)